# Chapter 6 – Exploring the Coordinator, Worker, and Delegator Approach
---

Install dependencies

In [ ]:
!pip install crewai langchain-openai

In [3]:
import getpass
import os

api_key = getpass.getpass(prompt="Enter OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = api_key

### Role-based agents

Role-based agents within the CWD (Coordinator, Worker, and Delegator) model for a travel planner.

# CrewAI implementation
---

In [5]:
import os
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai.tasks import TaskOutput
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

llm = ChatOpenAI(model="gpt-4o-mini")

## Create the Agents

### Core Travel Workers

In [6]:
flight_booking_worker = Agent(
    role="Flight Booking Specialist",
    goal="Find and book the optimal flights for the traveler",
    backstory="""You are an experienced flight booking specialist with extensive knowledge of airlines, 
    routes, and pricing strategies. You excel at finding the best flight options balancing cost, 
    convenience, and comfort according to the traveler's preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

hotel_booking_worker = Agent(
    role="Hotel Accommodation Expert",
    goal="Secure the ideal hotel accommodations for the traveler",
    backstory="""You have worked in the hospitality industry for over a decade and have deep knowledge 
    of hotel chains, boutique accommodations, and local lodging options worldwide. You're skilled at 
    matching travelers with accommodations that meet their budget, location preferences, and amenity requirements.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

activity_planning_worker = Agent(
    role="Activities and Excursions Planner",
    goal="Curate personalized activities and experiences for the traveler",
    backstory="""You're a well-traveled activities coordinator with insider knowledge of attractions, 
    tours, and unique experiences across numerous destinations. You're passionate about creating 
    memorable itineraries that align with travelers' interests, whether they seek adventure, culture, 
    relaxation, or culinary experiences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

transportation_worker = Agent(
    role="Local Transportation Coordinator",
    goal="Arrange efficient and convenient local transportation",
    backstory="""You specialize in local transportation logistics across global destinations. Your expertise 
    covers public transit systems, private transfers, rental services, and navigation, ensuring travelers 
    can move smoothly between destinations and activities.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

### Management Agents 

The Delegator and Coordinator agents

In [8]:
delegator_agent = Agent(
    role="Travel Planning Delegator",
    goal="Effectively distribute travel planning tasks to specialized workers",
    backstory="""You are an expert project manager with a talent for breaking down travel planning into 
    component tasks and assigning them to the right specialists. You understand each worker's strengths 
    and ensure they have the information needed to excel. You track progress, resolve bottlenecks, and 
    ensure all elements of the trip are properly addressed.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

coordinator_agent = Agent(
    role="Travel Planning Executive",
    goal="Ensure cohesive travel plans and maintain high customer satisfaction",
    backstory="""A seasoned travel industry veteran with 15 years of experience in luxury travel planning 
    and project management. Known for orchestrating seamless multi-destination trips for high-profile clients 
    and managing complex itineraries across different time zones and cultures. Expert in crisis management 
    and adaptive planning.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

## Define tasks for each agent

### Tasks for the workers

In [9]:
flight_search_task = Task(
    description="""
    Search for and recommend flight options based on the traveler's preferences.
    The traveler is looking to go from {origin} to {destination} departing around {departure_date} 
    and returning around {return_date}. Their priorities are: {flight_priorities}.
    
    Provide the following details for at least 2-3 options:
    1. Airline and flight numbers
    2. Departure and arrival times
    3. Duration and any layovers
    4. Class of service and approximate price
    5. Advantages and disadvantages of each option
    
    Make a recommendation based on the traveler's stated preferences.
    """,
    agent=flight_booking_worker,
    expected_output="""A detailed report of flight options with a clear recommendation 
    and justification based on the traveler's preferences."""
)

hotel_search_task = Task(
    description="""
    Research and recommend hotel accommodations in {destination} for the dates {check_in_date} 
    to {check_out_date}. The traveler's preferences include: {hotel_preferences} with a budget of 
    approximately {hotel_budget} per night.
    
    Provide the following details for at least 3 options:
    1. Hotel name, location, and star rating
    2. Room types available and recommended option
    3. Price per night and total for the stay
    4. Key amenities and unique features
    5. Proximity to attractions or points of interest
    6. Guest ratings and reviews summary
    
    Make a recommendation based on the traveler's stated preferences.
    """,
    agent=hotel_booking_worker,
    expected_output="""A detailed list of hotel options with comprehensive details 
    and a clear recommendation based on value, location, and the traveler's preferences."""
)

activity_planning_task = Task(
    description="""
    Create a daily activities plan for a {trip_duration} day trip to {destination}. The traveler's 
    interests include: {activity_interests} and they prefer to have {activity_pace} (busy/relaxed) days.
    
    For each day, provide:
    1. Morning, afternoon, and evening activity recommendations
    2. Estimated duration and cost for each activity
    3. Any reservation requirements or tips
    4. Contingency options in case of weather or other issues
    5. Brief description of why each activity was selected
    
    Ensure the itinerary flows logically with appropriate travel time between activities and rest periods if needed.
    """,
    agent=activity_planning_worker,
    expected_output="""A day-by-day itinerary with balanced activities matching the traveler's 
    interests and pace preferences, including timing, costs, and booking information."""
)

transportation_planning_task = Task(
    description="""
    Plan local transportation for the traveler in {destination} for their {trip_duration} day stay.
    They need transportation: 
    1. From the airport to their accommodation
    2. Between daily activities as outlined in the activities plan
    3. From their accommodation back to the airport
    
    The traveler's preferences for local transportation include: {transportation_preferences}
    
    Provide details on:
    1. Recommended transportation methods for each segment (public transit, taxi, rideshare, private transfer, car rental)
    2. Estimated costs for each option
    3. Booking information or apps needed
    4. Any local transportation passes that might provide value
    5. Key tips for navigating the local transportation system
    """,
    agent=transportation_worker,
    expected_output="""A comprehensive transportation plan covering all necessary transfers 
    during the trip with booking information, costs, and local transportation tips."""
)

### Tasks for the management agents (coordinator and delegator)

In [10]:
delegation_task = Task(
    description="""
    Review the traveler's requirements and preferences and delegate specific tasks to each specialized worker.
    
    Traveler Information:
    - Name: {traveler_name}
    - Trip Purpose: {trip_purpose}
    - Origin: {origin}
    - Destination: {destination}
    - Departure Date: {departure_date}
    - Return Date: {return_date}
    - Number of Travelers: {num_travelers}
    - Overall Budget: {overall_budget}
    - Special Requirements: {special_requirements}
    
    For each worker, create clear instructions with all necessary details they need to complete their tasks:
    1. Flight Booking Worker: Provide details on origin, destination, dates, and preferences.
    2. Hotel Booking Worker: Provide location, dates, budget, and accommodation preferences.
    3. Activity Planning Worker: Provide duration, interests, pace preferences, and any must-see attractions.
    4. Transportation Worker: Provide local transportation preferences and requirements.
    
    Create clear deliverables for each worker and establish deadlines.
    """,
    agent=delegator_agent,
    expected_output="""Detailed delegation instructions for each worker with all necessary information 
    and clear deliverables, formatted as a set of assignment briefs."""
)

coordination_task = Task(
    description="""
    Review all deliverables from the travel planning workers and create a cohesive final travel plan.
    
    Inputs:
    - Flight Booking Recommendations: {flight_recommendations}
    - Hotel Booking Recommendations: {hotel_recommendations}
    - Activities Plan: {activities_plan}
    - Transportation Plan: {transportation_plan}
    
    Your tasks:
    1. Ensure all elements of the plan work together (flights, hotel, activities, transportation)
    2. Identify and resolve any conflicts or gaps in the plan
    3. Create a day-by-day integrated itinerary
    4. Add essential details like contact information, confirmation numbers, and emergency contacts
    5. Format the plan in a user-friendly manner with clear sections
    6. Include a budget summary and payment schedule if applicable
    7. Add recommendations for travel insurance, packing, and destination-specific tips
    
    The final travel plan should be comprehensive yet easy to navigate and reference during the trip.
    """,
    agent=coordinator_agent,
    expected_output="""A complete, integrated travel plan with a day-by-day itinerary, 
    all booking details, contact information, and helpful travel tips. The plan should be 
    well-organized and user-friendly."""
)

#### Sample data for testing

In [11]:
# Define example traveler profile for testing
traveler_profile = {
    "traveler_name": "Alex Johnson",
    "trip_purpose": "Anniversary celebration",
    "origin": "New York City",
    "destination": "Paris, France",
    "departure_date": (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d"),
    "return_date": (datetime.now() + timedelta(days=37)).strftime("%Y-%m-%d"),
    "check_in_date": (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d"),
    "check_out_date": (datetime.now() + timedelta(days=37)).strftime("%Y-%m-%d"),
    "trip_duration": 7,
    "num_travelers": 2,
    "overall_budget": "$8,000",
    "hotel_budget": "$300",
    "flight_priorities": "Direct flights preferred, economy plus or business class, afternoon departure if possible",
    "hotel_preferences": "Boutique hotel in central location, preferably with a view of city landmarks, king bed",
    "activity_interests": "Fine dining, art museums, historical sites, wine tasting, evening entertainment",
    "activity_pace": "Moderate pace with some relaxation time built in",
    "transportation_preferences": "Mix of walking and public transit, with occasional taxis for evening outings",
    "special_requirements": "Need dinner reservations at a romantic restaurant for their anniversary night"
}

# Sample outputs for coordination
sample_outputs = {
    "flight_recommendations": "Option 1: Delta Airlines DL490/DL255, NYC-JFK to Paris-CDG, Departure 6:30 PM...",
    "hotel_recommendations": "Recommended: Hotel Particulier Montmartre, 4-star boutique hotel, €280/night...",
    "activities_plan": "Day 1: Morning - Arrive and check in, Afternoon - Light stroll along Seine...",
    "transportation_plan": "Airport to Hotel: Take RER B train from CDG to Saint-Michel Notre-Dame station..."
}

### Define the CrewAI workflow

In [ ]:
def travel_planning_workflow(traveler_info, process_type=Process.sequential):
    """
    Execute the travel planning workflow with CrewAI
    
    Args:
        traveler_info (dict): Dictionary containing traveler information and preferences
        process_type (Process): CrewAI process type (sequential or hierarchical)
        
    Returns:
        str: The final travel plan
    """
    
    # Set up the tasks with traveler information
    delegation_task_with_info = Task(
        description=delegation_task.description.format(**traveler_info),
        agent=delegator_agent,
        expected_output=delegation_task.expected_output
    )
    
    # Create a crew for delegation
    delegation_crew = Crew(
        agents=[delegator_agent, flight_booking_worker, hotel_booking_worker, 
                activity_planning_worker, transportation_worker],
        tasks=[delegation_task_with_info],
        verbose=True,
        process=process_type
    )
    
    # Execute delegation and get worker assignments
    delegation_result = delegation_crew.kickoff()
    print("\n\n=== Delegation Phase Complete ===\n\n")
    
    # Now we'd normally execute each worker's task with their specific instructions
    # For this example, we'll use placeholders for their outputs
    
    # Set up the coordination task with all results
    coordination_inputs = {**traveler_info, **sample_outputs}
    
    coordination_task_with_results = Task(
        description=coordination_task.description.format(**coordination_inputs),
        agent=coordinator_agent,
        expected_output=coordination_task.expected_output
    )
    
    # Create a crew for coordination
    coordination_crew = Crew(
        agents=[coordinator_agent],
        tasks=[coordination_task_with_results],
        verbose=True,
        process=process_type
    )
    
    # Execute coordination and get final plan
    final_plan = coordination_crew.kickoff()
    
    return final_plan

In [ ]:
# Function to run a full workflow
def run_full_workflow(traveler_info):
    """
    Run the complete workflow with actual tasks for each worker
    
    Args:
        traveler_info (dict): Dictionary containing traveler information and preferences
        
    Returns:
        str: The final travel plan
    """
    # Create flight task with traveler info
    flight_task = Task(
        description=flight_search_task.description.format(**traveler_info),
        agent=flight_booking_worker,
        expected_output=flight_search_task.expected_output
    )
    
    # Create hotel task with traveler info
    hotel_task = Task(
        description=hotel_search_task.description.format(**traveler_info),
        agent=hotel_booking_worker,
        expected_output=hotel_search_task.expected_output
    )
    
    # Create activities task with traveler info
    activities_task = Task(
        description=activity_planning_task.description.format(**traveler_info),
        agent=activity_planning_worker,
        expected_output=activity_planning_task.expected_output
    )
    
    # Create transportation task with traveler info
    transportation_task = Task(
        description=transportation_planning_task.description.format(**traveler_info),
        agent=transportation_worker,
        expected_output=transportation_planning_task.expected_output
    )
    
    # Create a crew with all worker agents and their tasks
    worker_crew = Crew(
        agents=[flight_booking_worker, hotel_booking_worker, 
                activity_planning_worker, transportation_worker],
        tasks=[flight_task, hotel_task, activities_task, transportation_task],
        verbose=True,
        process=Process.sequential
    )
    
    # Execute all worker tasks
    worker_results = worker_crew.kickoff()
    
    # Parse the results - in a real application, you would extract each result properly
    # Here we'll simulate the structure
    worker_outputs = {
        "flight_recommendations": worker_results[0] if isinstance(worker_results, list) else "Flight recommendations provided by Flight Booking Specialist",
        "hotel_recommendations": worker_results[1] if isinstance(worker_results, list) else "Hotel recommendations provided by Hotel Accommodation Expert",
        "activities_plan": worker_results[2] if isinstance(worker_results, list) else "Activities plan provided by Activities and Excursions Planner",
        "transportation_plan": worker_results[3] if isinstance(worker_results, list) else "Transportation plan provided by Local Transportation Coordinator"
    }
    
    # Set up the coordination task with the actual results
    coordination_inputs = {**traveler_info, **worker_outputs}
    
    coordination_task_with_results = Task(
        description=coordination_task.description.format(**coordination_inputs),
        agent=coordinator_agent,
        expected_output=coordination_task.expected_output
    )
    
    # Create a crew for coordination
    coordination_crew = Crew(
        agents=[coordinator_agent],
        tasks=[coordination_task_with_results],
        verbose=True,
        process=Process.sequential
    )
    
    # Execute coordination and get final plan
    final_plan = coordination_crew.kickoff()
    
    return final_plan

In [ ]:
# Simulate a hierarchical delegation workflow
def hierarchical_workflow(traveler_info):
    """
    Run a hierarchical workflow with the Coordinator delegating to the Delegator,
    who then delegates to workers
    
    Args:
        traveler_info (dict): Dictionary containing traveler information and preferences
        
    Returns:
        str: The final travel plan
    """
    # First, create a high-level task for the Coordinator
    high_level_task = Task(
        description=f"""
        Create a comprehensive travel plan for {traveler_info['traveler_name']}'s trip to {traveler_info['destination']}.
        
        Trip Details:
        - Purpose: {traveler_info['trip_purpose']}
        - Origin: {traveler_info['origin']}
        - Destination: {traveler_info['destination']}
        - Dates: {traveler_info['departure_date']} to {traveler_info['return_date']}
        - Number of Travelers: {traveler_info['num_travelers']}
        - Overall Budget: {traveler_info['overall_budget']}
        
        Delegate tasks as needed to the Travel Planning Delegator, who can further delegate 
        specialized tasks to the appropriate workers.
        
        Your final deliverable should be a complete, integrated travel plan covering all 
        aspects of the trip from departure to return.
        """,
        agent=coordinator_agent,
        expected_output="A comprehensive travel plan document"
    )
    
    # Set up all agents in a hierarchical crew
    hierarchical_crew = Crew(
        agents=[
            coordinator_agent,
            delegator_agent,
            flight_booking_worker,
            hotel_booking_worker,
            activity_planning_worker,
            transportation_worker
        ],
        tasks=[high_level_task],
        verbose=True,
        process=Process.hierarchical
    )
    
    # Execute the hierarchical workflow
    final_plan = hierarchical_crew.kickoff()
    
    return final_plan

In [16]:
# Execute the workflow in a Jupyter cell
if __name__ == "__main__":
    print("Starting Travel Planning Workflow...")
    # Choose which workflow to run:
    
    # Option 1: Simple sequential delegation
    result = travel_planning_workflow(traveler_profile, Process.sequential)
    
    # Option 2: Full workflow with all workers executing tasks
    # result = run_full_workflow(traveler_profile)
    
    # Option 3: Hierarchical workflow
    # result = hierarchical_workflow(traveler_profile)
    
    print("\n=== FINAL TRAVEL PLAN ===\n")
    print(result)

Starting Travel Planning Workflow...


ValidationError: 1 validation error for Crew
verbose
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/bool_parsing

# OpenAI Agent SDK implementation
---